# Libs. to import

In [ ]:
import time
from qiskit.circuit.library import RealAmplitudes
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA,SLSQP
from qiskit_ibm_runtime import EstimatorV2
from qiskit.primitives import Estimator
from qiskit.circuit.library import EfficientSU2
from qiskit_aer import Aer
from qiskit_aer import AerSimulator, QasmSimulator, StatevectorSimulator
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, runtime_job
from qiskit_nature.second_q.hamiltonians import HeisenbergModel
from qiskit_nature.second_q.problems import LatticeModelProblem
from qiskit_nature.second_q.mappers import *
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit import visualization
from qiskit_algorithms import NumPyMinimumEigensolver
import numpy as np
from math import pi
import rustworkx as rx
from qiskit_nature.second_q.hamiltonians.lattices import ( BoundaryCondition, 
Lattice,
LatticeDrawStyle,
LineLattice)
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
import pandas as pd
import ace_tools_open as tools

# IBM Account Loading

In [ ]:
QiskitRuntimeService.save_account(token="AIP",channel = 'ibm_quantum' ,overwrite=True)
service = QiskitRuntimeService()

# Fininding availabe quantum processors according to the number of spins

In [ ]:
# Set the number of spins (nodes)
num_nodes =2 # Start with 8 qubits for real quantum device

# List available backends
backends = service.backends()
for backend in backends:
    print(backend.name)

# Select a real quantum backend with at least num_nodes qubits
backend = service.least_busy(min_num_qubits=num_nodes)
print(f"Using backend: {backend.name}")

In [ ]:
#if need to run the code on simulator not real quantum computer uncomment the following step
#backend = AerSimulator(method='matrix_product_state')
backend = StatevectorSimulator()
#backend = QasmSimulator()
backend

# Preparing the Systsm

In [ ]:
no_Spins = 20

# Define The lattice 
line_latticec = LineLattice(num_nodes=no_Spins, boundary_condition=BoundaryCondition.PERIODIC)

# Construct the Heisenberg Hamiltonian

interaction_coefficients = (1,1,1) #J on x,y and z axis 
external_magntic_field = (0,0,0) # magnetic field

heisenberg_model = HeisenbergModel (line_latticec, interaction_coefficients, external_magntic_field)

#Generate spin Hamiltonian to map it on the qubits 
spin_ham = heisenberg_model.second_q_op()


print (f'the system is defined, number of spins:  {no_Spins} \n Hiesenberg Hamiltonian: {heisenberg_model.interaction_matrix()} \n Spin Hamiltonian: {spin_ham}')

# Mapping the system into quantum circuit 

In [ ]:
mapper = LogarithmicMapper()
qubit_op = mapper.map(spin_ham)

print (f'The number of qubits in the qubit operator is {qubit_op.num_qubits}')

# Finding the Ground State Energy Using Viriational Quantum Eigensolver (VQE)
VQE requires three inputs to find the miniumum eigenvalue (ground state energy): \
1- Estimator\
2- Ansatz\
3- Optimizer 

In [ ]:
# These steps to prepare the system for VQE 
estimator = Estimator(options={"backend":backend, "shots":1024})
ansatz = EfficientSU2(qubit_op.num_qubits, reps=2)
#print(f'number of qubits in ansatz is: {ansatz}')
#This step to make the quantum circuit most efficient for the quantum process we chose previously
transpiled_ansatz = transpile (ansatz,backend=backend,optimization_level=3)
optimizer = SLSQP(maxiter=200) 

print(f"Number of qubits in the qubit operator: {qubit_op.num_qubits}")
print(f"Number of qubits in the ansatz: {ansatz.num_qubits}") 


In [ ]:
vqe = VQE(estimator, ansatz , optimizer)

start_time = time.time()
#Compute the minimum eigenvalue 
result = vqe.compute_minimum_eigenvalue(qubit_op)
end_time=time.time()

time_taken = end_time - start_time 

total_energy= result.eigenvalue.real
energy_per_spin = total_energy/no_Spins
print (total_energy)
print (energy_per_spin)
print (f"energy calcaulation time is: {time_taken} seconds")